In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_moons
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np

#visualization
import matplotlib.pyplot as plt
import seaborn as sns

#preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer

#Classification Algorithms
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from tqdm.notebook import tqdm
from sklearn.model_selection import GridSearchCV

In [2]:
!pip install delayed

In [3]:
!pip install lightgbm

In [4]:
!pip install xgboost

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
train = pd.read_csv("delivery/train.csv", index_col='ID')
X_test = pd.read_csv("delivery/test.csv", index_col='ID')

In [7]:
train['Customer_care_calls'] = train['Customer_care_calls'].fillna(4.0)
train = train.fillna(0)

In [8]:
y_train = train.iloc[:,-1:]
X_train = train.iloc[:,:-1]

In [9]:
X_train=X_train.drop(['Gender'],axis=1)
X_test=X_test.drop(['Gender'],axis=1)

In [10]:
X_train.loc[(X_train.Weight_in_gms=='?'),'Weight_in_gms'] =3656
X_test.loc[(X_test.Weight_in_gms=='?'),'Weight_in_gms'] =3656
X_train['Weight_in_gms'] = X_train['Weight_in_gms'].astype(int)
X_test['Weight_in_gms'] = X_test['Weight_in_gms'].astype(int)

In [11]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [12]:
a = set(X_train.columns) - set(X_test.columns)
a = list(a)
for i in tqdm(range(len(a))):
    X_test[a[i]]= 0
    tmp = X_train[a[i]]
    X_train.drop(a[i],axis =1, inplace=True)
    X_train = pd.concat([X_train,tmp],axis=1)

In [13]:
b = set(X_test.columns) - set(X_train.columns)
b = list(b)
for i in tqdm(range(len(b))):
    X_train[b[i]]= 0
    tmp = X_test[b[i]]
    X_test.drop(b[i],axis =1, inplace=True)
    X_test = pd.concat([X_test,tmp],axis=1)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()   
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# import hyperopt
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
# reg_candidate = [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 5, 10, 100]

# space={'max_depth': hp.quniform("max_depth", 3, 10, 1),
#        'gamma': hp.uniform ('gamma', 0, 1),
#        'learning_rate': hp.quniform ('learning_rate', 0.01, 0.05, 0.005),
#        'reg_alpha' : hp.choice('reg_alpha', reg_candidate),
#        'reg_lambda' : hp.choice('reg_lambda', reg_candidate),
#        'subsample': hp.quniform('subsample', 0.6, 1, 0.05),
#        'colsample_bytree' : hp.quniform('colsample_bytree', 0.6, 1, 0.05),
#        'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
#        'n_estimators': hp.quniform('n_estimators', 300, 1500, 100)
#       }

In [16]:
# from xgboost import XGBClassifier
# from sklearn.metrics import mean_squared_error
# def hyperparameter_tuning(space):
#     model=XGBClassifier(n_estimators =int(space['n_estimators']), 
#                        max_depth = int(space['max_depth']), 
#                        gamma = space['gamma'],
#                        learning_rate = space['learning_rate'],
#                        reg_alpha = space['reg_alpha'],
#                        reg_lambda = space['reg_lambda'],
#                        subsample = space['subsample'],
#                        colsample_bytree = space['colsample_bytree'], 
#                        min_child_weight = int(space['min_child_weight']),
#                        random_state=42, 
#                       )
    
#     evaluation = [(X_train, y_train), (X_test, y_test)]
    
#     model.fit(X_train, y_train,
#               eval_set=evaluation, 
#               eval_metric="error",
#               early_stopping_rounds=30,
#               verbose=0)

#     pred = model.predict(X_test)
#     error = mean_squared_error(y_test,pred)
       
#     #change the metric if you like
#     return {'loss':error, 'status': STATUS_OK, 'model': model}



In [17]:
# trials = Trials()
# best = fmin(fn=hyperparameter_tuning,
#             space=space,
#             algo=tpe.suggest,
#             max_evals=100,
#             trials=trials)
# best['max_depth'] = int(best['max_depth'])
# best['min_child_weight'] = int(best['min_child_weight'])
# best['n_estimators'] = int(best['n_estimators'])
# best['reg_alpha'] = reg_candidate[int(best['reg_alpha'])]
# best['reg_lambda'] = reg_candidate[int(best['reg_lambda'])]
# best['random_state'] = 123
# print (best)

In [18]:
# from sklearn.metrics import f1_score
# def print_all(model,X_train,X_test,y_train,y_test):
#     model.fit(X_train,y_train)
#     score = model.score(X_test,y_test)
#     pred = model.predict(X_test)
#     f1 = f1_score(pred,y_test)
#     print("f1 : {:.5f}, Score : {:.3f}".format(f1,score))
    

In [47]:
xg = XGBClassifier( n_estimators  = 600,
                    learning_rate = 0.01, 
                    max_depth     = 4,
                    random_state  = 11,
                   # min_child_weight= 11, 
                    objective= 'binary:logistic', 
                    use_label_encoder = False )

In [28]:
!pip install catboost

In [29]:
from catboost import CatBoostClassifier
catboost = CatBoostClassifier(
    iterations=500,
    random_seed=42,
    depth=4,
    leaf_estimation_iterations=8,
    learning_rate = 0.01,
    logging_level= 'Silent'
)

In [30]:
catboost.fit( X_train, y_train )
catboost.score( X_train, y_train )

0.683383340477211

In [48]:
xg.fit( X_train, y_train )
xg.score( X_train, y_train )

[12:30:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7002428918416916

In [25]:
pre=xg.predict(X_test)
result=pd.read_csv('delivery/sampleSubmission.csv')
result['Reached.on.Time_Y.N']=pre
result.to_csv('sampleSubmission_02.csv',index = False)